# Circle detector exam project

### Python Projektformulering

#### Gruppe: Disgusted Obligation

#### Medlemmer: Jesper Hingelbjerg, Albert Scheel Vandel

Vores ide til et projekt er kameratracking, hvor man skal finde et objekt ud fra et, flere, fokuspunkter og bedømme afstanden, positionen og retningen af objektet, heriblandt når objektet bevæger sig, og sætte det ind i et koordinatsystem. Kameraet vil betragte objektet fra fugleperspektiv. Ideen er at tracke eksempelvis en, på objektet, optegnet cirkel med en viser, radiusstreg. Når cirklen deformeres eller på en eller anden vis skrumper, kan man således se at der er et skift i afstand. Hvis viseren peger i en anden retning, kan man se at der er tale om retningsskift. For at genkende cirklen med viseren vil vi benytte os af object recognition, og vi vil sikkert også bruge noget movement detection når objektet bevæger sig. Desuden kunne vi godt tænke os at mappe en sti ud fra hvor objektet hidtil har været. 

### Vores projekt delt op i tasks:
#### -Kunne finde en cirkel i kameraet.
#### -Kunne finde afstanden af en cirklen ud fra dens størrelse i kameraet.
#### -Kunne tegne en sti efter objektet.
#### -Kunne finde ud af i hvilken retning objektet bevæger sig ved at tracke positionen af objektet.
#### -Kunne finde objektets orientering ud fra viseren.
#### -Kombinere alt dette.


In [7]:
import numpy as np
import cv2
import sys

# I have measured the focal length of the webcam used, with the focalLengthCalc.ipynb notebook.
focalLength = 530
circleWidth = 10
video_capture = cv2.VideoCapture(0)
w, h = (1240, 720)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, w)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, h)

while True:
    
#     This grabs the current frame from the webcam
    ret, frame = video_capture.read()
    if ret == True:
 
        grayRaw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#         gray = cv2.medianBlur(grayRaw, 5)
#         grayGaus = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,-12)
        gray = cv2.Canny(grayRaw,75,150)
        rows = gray.shape[0]
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, rows / 8,
                               param1=100, param2=30,
                               minRadius=1, maxRadius=300)
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                # circle center where i[0] is x and i[1] is y on the frame
                center = (i[0], i[1])
                #circle radius.
                radius = i[2]
                
                #this sets the base point of where the circle is found.
                #they are converted to int's because of overflow warnings.
                lines_x = int(center[0] - radius)
                lines_y = int(center[1] - radius)
                
                # Begins line detection.
                lines_to_find = grayRaw[lines_x:lines_x + (radius * 2),lines_y - radius:lines_y + (radius * 2)]
                minLineLength = 10
                maxLineGap = 100
                lines = cv2.HoughLinesP(lines_to_find,1,np.pi/180,minLineLength,maxLineGap)
                
#                 Drawing of the lines detected starts here.
#                 if lines is not None:
                   
#                    for x1,y1,x2,y2 in lines[0]:
#                        #if abs(x1 - x2) > radius and abs(y1 - y2) > radius:
#                        cv2.line(frame,(x1 + lines_x,y1 + lines_y),(x2+lines_x,y2+lines_y),(0,255,0),2)
                
                #cv2.circle(frame, center, 1, (0, 100, 100), 3)
                # This calculates the distance of the detected circle.
                # It then writes the calculated distance in the top left side
                if radius > 0:
                    sizeCalc = int((circleWidth * focalLength) / radius)
                    circleSize = "circleSize = " + str(sizeCalc) + "cm" 
                    cv2.putText(frame, circleSize,(30,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),2,cv2.LINE_AA)
                cv2.circle(frame, center, radius, (255, 0, 255), 3)
                #edges = cv2.Canny(gray[circles],50,150,apertureSize = 3)

        
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()